In [1]:
import json
import os
import folium
import pandas as pd
import webbrowser
import matplotlib.pyplot as plt
import base64
import random
import chardet

In [2]:
# ✅ 先检测文件编码
with open("/Users/qihanfei/Desktop/gofodashboard/TESTLAX2_23.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # 读取部分内容进行检测
    detected_encoding = result["encoding"]
    print(f"✅ 检测到的文件编码：{detected_encoding}")

# ✅ 使用检测到的编码读取 CSV
shipment_df = pd.read_csv("/Users/qihanfei/Desktop/gofodashboard/TESTLAX2_23.csv", encoding=detected_encoding, sep="\t")


✅ 检测到的文件编码：UTF-16


In [3]:
# ✅ 1. 自动检测 CSV 编码
with open("/Users/qihanfei/Desktop/gofodashboard/TESTLAX2_23.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # 读取部分内容进行检测
    detected_encoding = result["encoding"]
    print(f"✅ 检测到的文件编码：{detected_encoding}")

# ✅ 2. 读取 CSV，确保 `\t` 作为分隔符
shipment_df = pd.read_csv("/Users/qihanfei/Desktop/gofodashboard/TESTLAX2_23.csv", encoding=detected_encoding, sep="\t")

# ✅ 3. 查看列名，确保 `对象` 变成 `zipcode`
print("✅ 修正后 CSV 列名:", shipment_df.columns.tolist())

# ✅ 4. 处理 ZIP Code 列名
shipment_df.rename(columns={"对象": "zipcode"}, inplace=True)

# ✅ 5. 确保 ZIP Code 处理正确
shipment_df["zipcode"] = shipment_df["zipcode"].astype(str).str.zfill(5)

# ✅ 6. 处理数值数据
shipment_df["领件票数"] = pd.to_numeric(shipment_df["领件票数"], errors="coerce")
shipment_df["Final妥投量"] = pd.to_numeric(shipment_df["Final妥投量"], errors="coerce")

# ✅ 7. 计算 Final妥投率
shipment_df["Final妥投率"] = shipment_df["Final妥投量"] / shipment_df["领件票数"]
shipment_df["Final妥投率"].fillna(0, inplace=True)

# ✅ 8. 过滤掉 "合计" 行
shipment_df = shipment_df[shipment_df["zipcode"] != "合计"]

print("✅ 处理后数据预览:")
print(shipment_df.head())


✅ 检测到的文件编码：UTF-16
✅ 修正后 CSV 列名: ['日期', '口岸', '中心', '站点', '对象', '票数', '领件票数', 'Final妥投量', 'Final妥投率', '客退数', '客退率', '丢失被抢数', '丢失被抢率', '被抢数', '库内丢失数', '站点丢失数', '库内待签收数', '站点待签收数', '异常中', '断更中']
✅ 处理后数据预览:
         日期   口岸   中心       站点 zipcode  票数  领件票数  Final妥投量  Final妥投率  客退数  \
0  25/02/23  LAX  DEN  DEN-CBL   80002  25    25        25  1.000000    0   
1  25/02/23  LAX  DEN  DEN-CBL   80033  48    48        47  0.979167    0   
2  25/02/23  LAX  DEN  DEN-CBL   80202  31    31        30  0.967742    0   
3  25/02/23  LAX  DEN  DEN-CBL   80204  52    52        51  0.980769    0   
4  25/02/23  LAX  DEN  DEN-CBL   80211  58    58        57  0.982759    0   

   客退率  丢失被抢数  丢失被抢率  被抢数  库内丢失数  站点丢失数  库内待签收数  站点待签收数  异常中  断更中  
0    0      0  0.00%    0      0      0       0       0    0    0  
1    0      0  0.00%    0      0      0       0       1    1    1  
2    0      0  0.00%    0      0      0       0       1    1    1  
3    0      0  0.00%    0      0      0       0       1    1  

In [4]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager

# ✅ 自动查找可用的中文字体
def get_best_chinese_font():
    fonts = ["Microsoft YaHei", "SimHei", "WenQuanYi Zen Hei", "Arial Unicode MS"]
    for font in fonts:
        if font in [f.name for f in font_manager.fontManager.ttflist]:
            print(f"✅ 使用字体: {font}")
            return font
    print("⚠️ 未找到可用的中文字体，可能会显示乱码！")
    return "sans-serif"  # 兜底方案

# ✅ 设置 Matplotlib 中文字体
best_font = get_best_chinese_font()
matplotlib.rcParams["font.family"] = best_font
matplotlib.rcParams["axes.unicode_minus"] = False  # 处理负号问题

✅ 使用字体: Arial Unicode MS


In [5]:
# ✅ 计算各站点的合计数据
site_summary = shipment_df.groupby("站点").agg(
    total_tickets=("领件票数", "sum"),
    total_delivered=("Final妥投量", "sum")
).reset_index()
site_summary["Final妥投率"] = site_summary["total_delivered"] / site_summary["total_tickets"]
site_summary["Final妥投率"].fillna(0, inplace=True)  # 避免 NaN


# ✅ 确保 `site_summary` 存在
print("✅ 站点汇总数据：")
print(site_summary.head())

# ✅ 生成站点饼图
pie_chart_paths = {}

for _, row in site_summary.iterrows():
    site = row["站点"]
    delivered = row["total_delivered"]
    not_delivered = row["total_tickets"] - delivered

    labels = ["Final妥投量", "未妥投量"]
    sizes = [delivered, not_delivered]
    colors = ["green", "red"]

    plt.figure(figsize=(2, 2))
    plt.pie(sizes, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140)
    plt.axis("equal")

    # 保存为图片
    img_path = f"pie_chart_{site}.png"
    plt.savefig(img_path, bbox_inches="tight"
    , dpi=100)
    plt.close()

    # 转换为 base64 编码的 HTML 图片
    with open(img_path, "rb") as img_file:
        base64_str = base64.b64encode(img_file.read()).decode("utf-8")
        pie_chart_paths[site] = f'<img src="data:image/png;base64,{base64_str}" width="150px"/>'

print("✅ 站点饼图生成完成！")



✅ 站点汇总数据：
        站点  total_tickets  total_delivered  Final妥投率
0  DEN-CBL           1358             1324  0.974963
1  DEN-WDL           2054             1792  0.872444
2  DEN-XAE           7710             6542  0.848508
3  DEN-YON            948              894  0.943038
4  LAS-FTN            596              584  0.979866
✅ 站点饼图生成完成！


In [6]:
# ✅ 6. 提取运单涉及的 ZIP Code
shipment_zipcodes = shipment_df["zipcode"].unique().tolist()
print(f"✅ 运单涉及 {len(shipment_zipcodes)} 个唯一 ZIP Code")

# ✅ 7. 加载 ZIP Code GeoJSON 文件
geojson_path = "/Users/qihanfei/Desktop/gofodashboard/merged_zip_codes.geojson"  # 修改为你的文件路径

try:
    with open(geojson_path, "r", encoding="utf-8") as f:
        zipcode_geojson = json.load(f)
    print("✅ ZIP Code 边界文件加载成功！")
except FileNotFoundError:
    print(f"❌ 文件 {geojson_path} 不存在！")
    exit()



✅ 运单涉及 1181 个唯一 ZIP Code
✅ ZIP Code 边界文件加载成功！


In [7]:
import json
import folium
import random

# ✅ 生成站点颜色
site_colors = {site: "#{:06x}".format(random.randint(0, 0xFFFFFF)) for site in site_summary["站点"].unique()}

# ✅ 仅保留运单中涉及的 ZIP Code，并添加统计数据
filtered_features = []

for feature in zipcode_geojson["features"]:
    zip_code = str(feature["properties"].get("ZCTA5CE10", "")).zfill(5)

    if zip_code in shipment_zipcodes:
        site_info = shipment_df[shipment_df["zipcode"] == zip_code].iloc[0]  # 获取站点信息
        site_name = str(site_info["站点"])  # 确保是 `str`
        site_data = site_summary[site_summary["站点"] == site_name].iloc[0]

        # 确保 `Final妥投率` 是 `float`
        site_data["Final妥投率"] = float(site_data["Final妥投率"])

        # 颜色：如果 Final妥投率 < 95%，则红色，否则站点颜色
        color = "red" if site_data["Final妥投率"] < 0.95 else site_colors.get(site_name, "blue")

        # 更新 GeoJSON 信息（确保所有数据类型正确）
        feature["properties"].update({
            "站点": site_name,
            "总领件票数": int(site_data["total_tickets"]),  # 确保 `int`
            "总妥投量": int(site_data["total_delivered"]),  # 确保 `int`
            "合计Final妥投率": f"{site_data['Final妥投率']:.2%}",  # 确保 `float`
            "PieChart": pie_chart_paths.get(site_name, "")
        })
        filtered_features.append(feature)

filtered_geojson = {
    "type": "FeatureCollection",
    "features": filtered_features
}

print(f"✅ 过滤后，地图上仅显示 {len(filtered_features)} 个 ZIP Code")

# ✅ 计算地图中心（默认用美国中心点）
us_map = folium.Map(location=[39.8283, -98.5795], zoom_start=5, tiles="cartodbpositron")

# ✅ 颜色映射
def zip_code_style(feature):
    """根据站点 Final妥投率 设定颜色"""
    final_rate = float(feature["properties"].get("合计Final妥投率", "0%").strip("%")) / 100
    color = "red" if final_rate < 0.95 else site_colors.get(feature["properties"]["站点"], "blue")
    return {"fillColor": color, "color": color, "weight": 1.5, "fillOpacity": 0.4}

# ✅ 添加 ZIP Code 区域
folium.GeoJson(
    filtered_geojson,
    name="Filtered ZIP Code Boundaries",
    style_function=zip_code_style,
    tooltip=folium.GeoJsonTooltip(
        fields=["ZCTA5CE10", "站点", "总领件票数", "总妥投量", "合计Final妥投率", "PieChart"],
        aliases=["ZIP Code:", "站点:", "总领件票数:", "总妥投量:", "合计Final妥投率:", "妥投率饼图:"],
        localize=True
    )
).add_to(us_map)

# ✅ 保存地图
output_file = "shipment_site_map.html"
us_map.save(output_file)

print(f"📍 生成地图（按站点颜色标记）：{output_file}")


✅ 过滤后，地图上仅显示 1167 个 ZIP Code
📍 生成地图（按站点颜色标记）：shipment_site_map.html


In [ ]:
# ✅左上角按钮：查看低妥投率站点
summary_html = site_summary[site_summary["Final妥投率"] < 0.95].to_html(classes="table table-striped", border=0)
summary_file = "summary.html"
with open(summary_file, "w", encoding="utf-8") as f:
    f.write(f"<h2>📢 妥投率 < 95% 的站点</h2>{summary_html}")

button_html = f"""
<div style="
    position: fixed;
    top: 10px; left: 10px;
    z-index: 1000;
    background-color: white;
    padding: 10px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
">
    <a href="{summary_file}" target="_blank" style="font-size:14px; color:red; text-decoration:none;">
        📊 查看妥投率不达标站点
    </a>
</div>
"""

us_map.get_root().html.add_child(folium.Element(button_html))



In [ ]:
# ✅保存 & 打开地图
output_file = "shipment_site_map.html"
us_map.save(output_file)
webbrowser.open(f"file://{os.path.abspath(output_file)}")

print(f"📍 生成地图（按站点颜色标记）：{output_file}")
print(f"📄 生成 Summary 报告（妥投率不达标）：{summary_file}")


📍 生成地图（按站点颜色标记）：shipment_site_map.html
📄 生成 Summary 报告（妥投率不达标）：summary.html
